# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [39]:
import sqlite3
import pandas as pd
import numpy as np
import datetime as dt
import os

pd.set_option('display.max_column', None)

In [40]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [41]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)
match_data = pd.read_sql("SELECT * FROM Match;", db)
leagues_data = pd.read_sql("SELECT * FROM League;", db)
player_attribute_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [42]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [43]:
players_180_190 = player_data[(player_data['height'] <= 190) & (player_data['height'] >= 180)].shape[0]

In [44]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [45]:
players_1980 = player_data[pd.to_datetime(player_data['birthday']).dt.year == 1980].shape[0]

In [46]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [47]:
highest_players = player_data.sort_values(by=['weight', 'player_name'], ascending=(False, True))\
    ['player_name'].head(10).to_list()

In [48]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [49]:
player_datetime = pd.to_datetime(player_data['birthday'])
years_born_players = list(player_datetime.dt.year[
                              (player_datetime.dt.year >= 1980) &
                              (player_datetime.dt.year <= 1990)]
                          .value_counts().to_dict().items())
years_born_players.sort()

In [50]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [165]:
players_adriano = player_data[player_data['player_name'].str.match(r'Adriano( .+)?')]
adriano_mean, adriano_std = players_adriano['height'].mean(), players_adriano['height'].std()

(182.118, 5.086346036203199)

In [52]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [53]:
dow_with_min_players_born =player_datetime.dt.day_name().value_counts().idxmin()

In [54]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [55]:
league_matches = match_data['league_id'].value_counts()
leagues_most_matches =  league_matches[league_matches == league_matches.max()].index

league_most_matches = leagues_data.loc[leagues_data['id'].isin(leagues_most_matches)]['name'].sort_values().values[0]

In [56]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [57]:
player_matches = pd.concat((match_data.filter(regex=r'home_player_\d'), match_data.filter(regex=r'away_player_\d')), axis=1)

player_matches_count = pd.DataFrame()
for column_name in player_matches:
    player_matches_count = pd.concat((player_matches_count, player_matches[column_name].value_counts()), axis=1)

max_matches_player = player_data[player_data['player_api_id'] == player_matches_count.sum(axis=1).idxmax()]['player_name'].values[0]

In [58]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [59]:
# вернуться и улучшить (сделать таблицу из 3-х колонок - пара и значение). А то гавно перегруженное

corr_characteristics = player_attribute_data.drop([
    'id', 'player_fifa_api_id', 'player_api_id', 'date',
    'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'
], axis=1).corr()

corr_char_values = corr_characteristics[corr_characteristics != 1].values.flatten()
corr_char_values = np.abs(corr_char_values[~np.isnan(corr_char_values)])
corr_char_values[::-1].sort()

raw_top_correlated_features = []
nlargest = 0
while len(raw_top_correlated_features) < 5:
    cell = corr_characteristics[corr_characteristics == corr_char_values[nlargest]].dropna(how='all').dropna(axis=1, how='all')
    column, row = cell.columns[0], cell.index[1]
    corr_feature = {column, row, float(corr_char_values[nlargest])}
    if corr_feature not in raw_top_correlated_features:
        raw_top_correlated_features.append(corr_feature)
    nlargest += 1
raw_top_correlated_features = list(map(tuple, raw_top_correlated_features))

top_correlated_features = []
for val1, val2, val3 in raw_top_correlated_features:
    if isinstance(val1, float):
        top_correlated_features.append((val2, val3))
    elif isinstance(val2, float):
        top_correlated_features.append((val1, val3))
    else:
        top_correlated_features.append((val1, val2))

In [60]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [61]:
players_characteristics = player_attribute_data.drop([
    'id', 'player_fifa_api_id',
    'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'
], axis=1)
players_characteristics['date'] = pd.to_datetime(players_characteristics['date'])

neymar_player_id = player_data[player_data['player_name'] == 'Neymar']['player_api_id'].values[0]
neymar_characteristics = players_characteristics[
    players_characteristics['player_api_id'] == neymar_player_id]\
    .sort_values(by=['date'], ascending=False).iloc[0]\
    .drop(['player_api_id', 'date'])

players_unique_characteristics = players_characteristics.sort_values(
    by=['player_api_id', 'date'], ascending=False)\
    .drop_duplicates(subset=['player_api_id'], keep='first').drop(['date'], axis=1)
players_neymar_distance = players_unique_characteristics.subtract(neymar_characteristics, axis=1).drop('player_api_id', axis=1) ** 2
players_neymar_distance['distance'] = players_neymar_distance.sum(axis=1) ** 0.5
players_neymar_distance = players_neymar_distance[players_neymar_distance['distance'] != 0].sort_values(by=['distance'])

neymar_similarities = [] # как то можно по красивому
for player_id in players_unique_characteristics.loc[players_neymar_distance.head().index]['player_api_id']:
    neymar_similarities.append(player_data[player_data['player_api_id'] == player_id]['player_name'].values[0])

In [62]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [63]:
bundesliga_id = leagues_data[leagues_data['name'] == 'Germany 1. Bundesliga']['id'].values[0]
borussia_bundesliga_2008_2009_matches = match_data[
    (match_data['home_team_api_id'] == team_data[team_data['team_long_name'] == 'Borussia Dortmund']['team_api_id'].values[0]) &
    (match_data['season'] == '2008/2009') & (match_data['league_id'] == bundesliga_id)].shape[0]

In [64]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [65]:
team_most_matches_bundesliga_2008_2009 = int((
        match_data[(match_data['league_id'] == bundesliga_id) & (match_data['season'] == '2008/2009')]['home_team_api_id'].value_counts() +
        match_data[(match_data['league_id'] == bundesliga_id) & (match_data['season'] == '2008/2009')]['away_team_api_id'].value_counts()).max())

In [66]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [67]:
arsenal_id = team_data[team_data['team_long_name'] == 'Arsenal']['team_api_id'].values[0]
arsenal_mathes_2015_2016 = match_data[(match_data['season'] == '2015/2016') & ((match_data['home_team_api_id'] == arsenal_id) | (match_data['away_team_api_id'] == arsenal_id))]
arsenal_home_matches = arsenal_mathes_2015_2016[(arsenal_mathes_2015_2016['home_team_api_id'] == arsenal_id)]
arsenal_away_matches = arsenal_mathes_2015_2016[(arsenal_mathes_2015_2016['away_team_api_id'] == arsenal_id)]

arsenal_won_matches_2015_2016 = arsenal_home_matches[arsenal_home_matches['home_team_goal'] > arsenal_home_matches['away_team_goal']].shape[0] + \
    arsenal_away_matches[arsenal_away_matches['home_team_goal'] < arsenal_away_matches['away_team_goal']].shape[0]

In [68]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [95]:
matches_season_2015_2016 = match_data[match_data['season'] == '2015/2016']

match_season_2015_2016_winners = matches_season_2015_2016.copy()
match_season_2015_2016_winners.loc[match_data['home_team_goal'] > match_data['away_team_goal'], 'winner_id'] = match_data['home_team_api_id']
match_season_2015_2016_winners.loc[match_data['home_team_goal'] < match_data['away_team_goal'], 'winner_id'] = match_data['away_team_api_id']
season_2015_2016_win_matches = match_season_2015_2016_winners['winner_id'].dropna().value_counts()
season_2015_2016_total_matches = (matches_season_2015_2016['home_team_api_id'].value_counts() +
                                  matches_season_2015_2016['away_team_api_id'].value_counts())

team_highest_winrate_2015_2016 = team_data[team_data['team_api_id'] == (season_2015_2016_win_matches / season_2015_2016_total_matches).idxmax()]['team_long_name'].values[0]

'SL Benfica'

In [96]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [148]:
match_data['date'] = pd.to_datetime(match_data['date'])
matches_england_2010_2011 = match_data[(
        match_data['league_id'] == leagues_data[leagues_data['name'] == 'England Premier League']['id'].values[0]) &
                                       (match_data['season'] == '2010/2011')]
teams_id_england_2010_2011 = matches_england_2010_2011['home_team_api_id'].value_counts().index

teams_match_gap = pd.DataFrame(index=teams_id_england_2010_2011, columns=['gap'])
teams_match_gap['gap'] = pd.to_datetime(teams_match_gap['gap'])

for team_id in teams_id_england_2010_2011:
    team_matches = matches_england_2010_2011[(
        matches_england_2010_2011['home_team_api_id'] == team_id
                                             ) | (
        matches_england_2010_2011['away_team_api_id'] == team_id
    )].sort_values(by=['date'])
    teams_match_gap.loc[team_id] = (team_matches['date'] - team_matches['date'].shift()).max()

highest_gap_england_2010_2011 = teams_match_gap['gap'].max().days

18

In [150]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [151]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")